# Toxic Comment Classification - EDA

In [ ]:
#load libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#load datasets
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

# Exploratory Data Analysis

In [ ]:
train.columns

Our target variables in here are classified as follows:
    * toxic
    * severe_toxic
    * obscene
    * threat
    * insult
    * identity_hate

In [ ]:
bad_tags = train.iloc[:, 2:].sum()

rowsums=train.iloc[:,2:].sum(axis=1)

train['clean']= (rowsums == 0)
binary = {True : 1, False : 0} #0 - bad comments, 1 - clean comments
train["clean"] = train["clean"].map(binary)

In [ ]:
x=train.iloc[:,2:].sum()
#plot
plt.figure(figsize=(8,4))
ax= sns.barplot(x.index, x.values, alpha=0.8)
plt.title("# per class")
plt.ylabel('# of Occurrences', fontsize=12)
plt.xlabel('Type ', fontsize=12)

Observations:
    1. Most of the comments are clean.
    2. Since classification is mostly clean, there might be a possible class imbalance. 

In [ ]:
train.sample(10)

Observations:
    1. A sample from our training set shows that classification of bad comments can be tagged into multple categories.
    2. Sentiment for clean comments range from neutral to positive.

# Does String Length Affect Sentiment Scores?

In [ ]:
train["comment_length"] = train.comment_text.str.len()

g = sns.FacetGrid(train, hue ="clean")
g.map(sns.distplot, "comment_length")
g.fig.set_size_inches(12, 6)


plt.legend()

Apparently, string length does not provide any useful information. Both clean and bad comments are spread out evenly.

# What Makes a Good/Bad Comment?

For this section, we'll check out what makes a good and a bad comment. We'll sample a few examples from our training set and try to find any patterns.

In [ ]:
#Clean Comment Sample

train[train["clean"] == 1].sample(10)

Observation:
    1. Sentiments on these comments range from neutral to positive.

In [ ]:
#Toxic Comment Sample

train[train["toxic"] == 1].sample(10)

It seems that toxic comments have an aggressive tone, and a highly negative sentiment.

In [ ]:
#Sample severe_toxic comments

train[train["severe_toxic"] == 1].sample(10)

Severe toxic comments, are usually associated with more tags and are linearly correlated obscene and insult comments.

In [ ]:
#Sample obscene comments

train[train["obscene"] == 1].sample(10)

Obscene comments are linearly correlated with insult comments.

In [ ]:
#Sample threat comments

train[train["threat"] == 1].sample(10)

For the threat comments are linearly correlated with toxic, obscene, and insult comments.

In [ ]:
#Sample insult comments

train[train["insult"] == 1].sample(10)

The same results have been found for obscene comments, which are highly correlated with insults as well as the toxic tag.

In [ ]:
#Sample identity_hate comments

train[train["identity_hate"] == 1].sample(10)

For our identity_hate comments, there is a linear correaltion with insult and toxic comments.

Let's provide a heatmap to check to find out linear correlations between each variable, and inspect if our observations for them are true.

In [ ]:
plt.figure(figsize = (12, 8))
sns.heatmap(train.iloc[:, 2:-1].corr(), annot = True)

1. We have confirmed our observations that both obscene and insult comments are highly linearly correlated with each other.
2. Also toxic comments are highly correlated with obscene and insult comments.

# What are the common attributes for each sample?

    - Another thing is that a lot of the toxic comments have cussing in them.
    - Bad comments have a lot of negative sentiment in them.
    - Negative comments are also have an aggressive tone.
    - Bad comments are usually classified into several tags.

# Sentiment Analysis

Since all observed bad comments are highly negative, let's check for the sentiment analysis for each of the classified comments.

For this section, I'm going to use the vaderSentiment library which outputs a compound sentiment between -1 to 1, where the former means that the comment is purely negative while the latter, otherwise.

In [ ]:
sentiment = SentimentIntensityAnalyzer()

In [ ]:
toxic_vs_clean = []

for index in train.index:
    toxic_vs_clean.append(sentiment.polarity_scores(train.iloc[index, 1]))

In [ ]:
data = pd.concat([pd.DataFrame(toxic_vs_clean), train["clean"]], axis =1)

In [ ]:
g = sns.FacetGrid(data, hue = "clean")
g.map(sns.distplot, "compound")
g.fig.set_size_inches(12, 6)

plt.legend()

Suprisingly, compound scores for both toxic and clean data have scores ranging from -1 to 1. Let's investigate them further.

Observations:
    1. There are clean comments that have highly negative compound score.
    2. Comments that have a high neutral rating, bare very little with the compound score.
    3. Likewise, large wights have been given to both the negative and positive.
    
We're gonna add two more variables, which are the ratio between the negative and nutral score, and the positive and neutral score.

In [ ]:
analyze = pd.DataFrame(toxic_vs_clean)

analyze["neu_neg"] = analyze["neu"]/(analyze["neg"] + 0.0001)
analyze["neu_pos"] = analyze["neu"]/(analyze["pos"] + 0.0001)

eda = pd.concat([analyze, train["clean"]], axis =1)

In [ ]:
fig, [ax1, ax2] = plt.subplots(ncols = 2, nrows = 1, figsize = (12, 6))

sns.regplot("neu_neg", "compound", data = eda, ax = ax1)
sns.regplot("neu_pos", "compound", data = eda, ax = ax2)

Observations:

    1. For the first graph, high ratios (neutral score is a lot higher than the negative score), the polarity of the setniment is high.
    2. Likewise, for the second graph (low positive scores and high neutral scores), have a low sentiment.